In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import chess
import numpy as np
from sklearn.preprocessing import LabelEncoder


def algebraic_to_uci(board, algebraic_move):
    try:
        uci_text = board.push_san(algebraic_move).uci()
        return uci_text
    except chess.MoveError:
        move = chess.Move.from_uci(algebraic_move)
        if move in board.legal_moves:
            board.push(move)
            return move.uci()
        else:
            print(f"Error converting algebraic move '{algebraic_move}' to UCI: Move is illegal.")
            return ""
    except Exception as e:
        print(f"Error converting algebraic move '{algebraic_move}' to UCI: {e}")
        return ""


class ChessCNN(nn.Module):
    def __init__(self, num_classes):
        super(ChessCNN, self).__init__()
        self.conv1 = nn.Conv2d(12, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Función para aplanar la representación del tablero
def flatten_board_representation(board):
    flattened_board = np.zeros((12, 8, 8), dtype=np.float32)
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            value = 1.0
            flattened_board[piece.piece_type - 1][chess.square_rank(square)][chess.square_file(square)] = value

    return flattened_board

def prepare_data_for_player(player_df, player):
    X = []
    Y = []

    for i in range(len(player_df['moves']) - 1):
        current_board = chess.Board()
        for move in player_df['moves'].iloc[i].split():
            algebraic_to_uci(current_board, move)
            flattened_board = flatten_board_representation(current_board)
            X.append(flattened_board)

            # El siguiente movimiento es la etiqueta de salida
            next_move = player_df['moves'].iloc[i + 1].split()[-1]
            Y.append(next_move)

    return X, Y


df = pd.read_csv('data/chess_data.csv')

# Identificar jugadores con más de 60 partidas
white_player_counts = df['white_id'].value_counts()
black_player_counts = df['black_id'].value_counts()

# Seleccionar jugadores que hayan jugado más de 60 partidas en total (blancas y negras)
selected_players = set((white_player_counts + black_player_counts)[(white_player_counts + black_player_counts) > 60].index)

# Entrenar modelos para jugadores seleccionados
models = {} 
test_sets = {}

for player in selected_players:
    print(f"Training model for player: {player}")

    player_df = df[(df['white_id'] == player) | (df['black_id'] == player)]
    X, Y = prepare_data_for_player(player_df, player)

 
    X_array = np.array(X)
    Y_array = np.array(Y)

    X_tensor = torch.tensor(X_array, dtype=torch.float32)

    label_encoder = LabelEncoder()
    Y_encoded = label_encoder.fit_transform(Y_array)
    num_classes = len(label_encoder.classes_)


    Y_tensor = torch.tensor(Y_encoded, dtype=torch.long)

  
    X_train, X_test, Y_train, Y_test = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=42)

    # Definir un conjunto de datos personalizado
    class ChessDataset(torch.utils.data.Dataset):
        def __init__(self, X, Y):
            self.X = X
            self.Y = Y

        def __len__(self):
            return len(self.X)

        def __getitem__(self, idx):
            return self.X[idx], self.Y[idx]


    train_dataset = ChessDataset(X_train, Y_train)
    test_dataset = ChessDataset(X_test, Y_test)

   
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


    model = ChessCNN(num_classes)

    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Entrenar el modelo
    num_epochs = 10
    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Evaluar el modelo
        model.eval()
        all_predicted = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_predicted)
        print(f"Epoch {epoch + 1}/{num_epochs}, Test Accuracy: {accuracy}")

    # Almacenar el modelo entrenado para el jugador
    models[player] = model

    # Guardar el modelo en un archivo
    model_filename = f"{player}_chess_model.pth"
    torch.save(model.state_dict(), model_filename)
    print(f"Modelo para el jugador {player} guardado en {model_filename}")

    # Guardar el conjunto de prueba en un archivo
    test_set_filename = f"{player}_test_set.pth"
    torch.save((X_test, Y_test), test_set_filename)
    print(f"Conjunto de prueba para el jugador {player} guardado en {test_set_filename}")

    # Guardar el conjunto de prueba en el diccionario
    test_sets[player] = (X_test, Y_test)

Training model for player: anakgreget


/home/angela/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/10, Test Accuracy: 0.9183246073298429
Epoch 2/10, Test Accuracy: 0.9392670157068063
Epoch 3/10, Test Accuracy: 0.9455497382198953
Epoch 4/10, Test Accuracy: 0.9476439790575916
Epoch 5/10, Test Accuracy: 0.9434554973821989
Epoch 6/10, Test Accuracy: 0.9539267015706806
Epoch 7/10, Test Accuracy: 0.9539267015706806
Epoch 8/10, Test Accuracy: 0.9570680628272251
Epoch 9/10, Test Accuracy: 0.956020942408377
Epoch 10/10, Test Accuracy: 0.9528795811518325
Modelo para el jugador anakgreget guardado en anakgreget_chess_model.pth
Conjunto de prueba para el jugador anakgreget guardado en anakgreget_test_set.pth
Training model for player: artem555
Epoch 1/10, Test Accuracy: 0.9583333333333334
Epoch 2/10, Test Accuracy: 0.9699612403100775
Epoch 3/10, Test Accuracy: 0.9651162790697675
Epoch 4/10, Test Accuracy: 0.9641472868217055
Epoch 5/10, Test Accuracy: 0.9709302325581395
Epoch 6/10, Test Accuracy: 0.9718992248062015
Epoch 7/10, Test Accuracy: 0.9718992248062015
Epoch 8/10, Test Accuracy: 

In [2]:
# Crear un diccionario para almacenar los resultados de cada jugador
player_results = {}

# Bucle para imprimir los resultados de cada jugador
for player in selected_players:
    print(f"Modelo para el jugador {player} guardado en {player}_chess_model.pth")
    print(f"Conjunto de prueba para el jugador {player} guardado en {player}_test_set.pth")

    # Calcular y mostrar el accuracy medio para cada jugador
    model = models[player]
    test_set = test_sets[player]

    X_test, Y_test = test_set
    test_dataset = ChessDataset(X_test, Y_test)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

    model.eval()
    all_predicted = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predicted)
    player_results[player] = accuracy  # Almacenar el accuracy del jugador

    print(f"Epoch {epoch + 1}/{num_epochs}, Test Accuracy: {accuracy}")

# Calcular y mostrar el accuracy medio global
print("\nTabla de Accuracy Medio:")
print("{:<20} {:<15}".format("Jugador", "Accuracy Medio"))
print("-" * 35)

# Calcular y mostrar el accuracy medio global
global_accuracy = np.mean(list(player_results.values()))
print("{:<20} {:<15.2f}".format("Media General", global_accuracy))
print("-" * 35)

# Imprimir el accuracy medio para cada jugador
for player, accuracy in player_results.items():
    print("{:<20} {:<15.2f}".format(player, accuracy))
    print("-" * 35)


Modelo para el jugador anakgreget guardado en anakgreget_chess_model.pth
Conjunto de prueba para el jugador anakgreget guardado en anakgreget_test_set.pth
Epoch 10/10, Test Accuracy: 0.9528795811518325
Modelo para el jugador artem555 guardado en artem555_chess_model.pth
Conjunto de prueba para el jugador artem555 guardado en artem555_test_set.pth
Epoch 10/10, Test Accuracy: 0.9709302325581395
Modelo para el jugador ducksandcats guardado en ducksandcats_chess_model.pth
Conjunto de prueba para el jugador ducksandcats guardado en ducksandcats_test_set.pth
Epoch 10/10, Test Accuracy: 0.9251207729468599
Modelo para el jugador saviter guardado en saviter_chess_model.pth
Conjunto de prueba para el jugador saviter guardado en saviter_test_set.pth
Epoch 10/10, Test Accuracy: 0.9220917822838848
Modelo para el jugador a_p_t_e_m_u_u guardado en a_p_t_e_m_u_u_chess_model.pth
Conjunto de prueba para el jugador a_p_t_e_m_u_u guardado en a_p_t_e_m_u_u_test_set.pth
Epoch 10/10, Test Accuracy: 0.9217557